# Bibliotecas

In [ ]:
from obterDados import obterSimboloPosicao, obterSimboloData
import numpy as np
import tensorflow as tf
import gymnasium as gym
import gym_anytrading
from stable_baselines3 import A2C
from json import dumps

# Obtendo dados de M5 e D1

In [ ]:
rawDataM5 = obterSimboloPosicao()[:-100*108]
rawDataM5

In [ ]:
rawDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataM5

In [ ]:
rawDataD1 = obterSimboloData(rawDataM5.index[-1], n=int(len(rawDataM5)/108)+200)
rawDataD1

In [ ]:
rawDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataD1

# Criando ambiente e testando aleatoriamente

In [ ]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=rawDataM5, df_daily=rawDataD1)

In [ ]:
# observation = env.reset()
# done = False
# cont = 0
# while not done:
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)

# info

# Treinando modelo com aprendizado por reforço

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback

modelo = A2C('MultiInputPolicy', env, verbose=1, tensorboard_log='logs/a2c/')

models_dir = 'models/A2C'
checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
modelo.learn(total_timesteps=len(rawDataM5)*200, callback=checkpointCallback)

### Continuar treinamento a partir de modelo salvo

In [ ]:
# from stable_baselines3.common.callbacks import CheckpointCallback

# models_dir = 'models/A2C'
# model_path = f"{models_dir}/rl_model_10340000_steps.zip"
# modelo = A2C.load(model_path, env=env)
# checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
# modelo.learn(total_timesteps=len(rawDataM5)*300, callback=checkpointCallback, reset_num_timesteps=False)

# Testando modelo treinado

In [8]:
testDataM5 = obterSimboloPosicao()[-(20*108+300):]
testDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
testDataD1 = obterSimboloData(testDataM5.index[-1], n=int(len(testDataM5)/108)+200)
testDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)

In [9]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=testDataM5, df_daily=testDataD1)

In [10]:
models_dir = 'models/A2C'
model_path = f"{models_dir}/rl_model_15880000_steps.zip"
modelo = A2C.load(model_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\aurel\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [11]:
observation = env.reset()
done = False
while not done:
    action = modelo.predict(observation)
    observation, reward, done, info = env.step(int(action[0]))
    # env.render()

info

{'profit': 0, 'step': 2421, 'trades': []}